In [1]:
#pip install opencv-python mediapipe pyautogui pygame

In [2]:
import cv2
import mediapipe as mp
import pyautogui
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Inicializa MediaPipe y sus módulos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Captura de video
cap = cv2.VideoCapture(0)

# Inicializa Pygame
pygame.init()
window_width, window_height = 640, 480
screen = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Dibuja con la Mano")

# Definir color del lápiz
pen_color = (0, 128, 255)  # Azul
pen_radius = 5  # Grosor del lápiz
indicator_color = (255, 0, 0)  # Rojo para el indicador
indicator_radius = 10  # Grosor del indicador

# Dimensiones de la pantalla para mover el mouse
screen_width, screen_height = pyautogui.size()

# Para almacenar si la mano está cerrada
is_drawing = False
drawing_points = []  # Lista para almacenar puntos de dibujo

# Función para determinar si la mano está cerrada
def is_hand_closed(landmarks):
    finger_tips = [mp_hands.HandLandmark.INDEX_FINGER_TIP,
                   mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
                   mp_hands.HandLandmark.RING_FINGER_TIP,
                   mp_hands.HandLandmark.PINKY_TIP]

    is_closed = True
    for tip in finger_tips:
        base = tip - 2  # El nudillo correspondiente
        if landmarks[tip].y < landmarks[base].y:  # Si la punta está por arriba del nudillo
            is_closed = False
    return is_closed

# Bucle principal
running = True
while running:
    # Capturar los eventos de Pygame
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Captura el frame de la cámara
    ret, frame = cap.read()
    if not ret:
        break

    # Convierte la imagen a RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Procesa el frame con MediaPipe
    results = hands.process(frame_rgb)

    # Limpiar la pantalla de Pygame
    screen.fill((255, 255, 255))  # Fondo blanco

    # Si detecta manos
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Dibuja los puntos de referencia en la mano
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Obtén la posición de la muñeca (landmark 0)
            x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * window_width)
            y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * window_height)

            # Dibuja el indicador en la posición de la muñeca
            pygame.draw.circle(screen, indicator_color, (x, y), indicator_radius)

            # Verifica si la mano está cerrada para comenzar a dibujar
            if is_hand_closed(hand_landmarks.landmark):
                is_drawing = True
                drawing_points.append((x, y))  # Agrega el punto actual a la lista de puntos
            else:
                is_drawing = False
                drawing_points.append(None)  # Añade None para indicar un cambio de dibujo

            # Invertir el movimiento: si la mano se mueve a la izquierda, el lápiz se mueve a la derecha
            mouse_x = window_width - x  # Inversión del movimiento horizontal
            mouse_y = y  # Mantener la misma altura

            # Mueve el mouse basándose en la posición invertida de la mano
            #pyautogui.moveTo(mouse_x, mouse_y)

    # Dibuja líneas entre los puntos almacenados
    for i in range(1, len(drawing_points)):
        if drawing_points[i] is not None and drawing_points[i - 1] is not None:
            pygame.draw.line(screen, pen_color, drawing_points[i - 1], drawing_points[i], pen_radius)

    # Dibuja la imagen de la cámara en modo espejo
    frame_flipped = cv2.flip(frame, 1)  # Invertir horizontalmente
    cv2.imshow("Detección de Manos", frame_flipped)  # Mostrar la imagen invertida

    # Actualiza la pantalla de Pygame
    pygame.display.flip()

    # Salir del bucle si se presiona la tecla Esc
    if cv2.waitKey(1) & 0xFF == 27:  # Esc para salir
        running = False

# Libera la cámara y cierra las ventanas
cap.release()
cv2.destroyAllWindows()
pygame.quit()


c:\Users\29tau\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
